### Business Case : Predict insurance amount based on historical data to reduce customer

In [2]:
train_data = spark.read.csv('train.csv',header=True, inferSchema=True)

In [4]:
test_data = spark.read.csv('test.csv',header=True, inferSchema=True)

In [6]:
train_data.show(5)

+---+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+--------+--------+--------+--------+--------+--------+--------+-------+-------+-------+--------+--------+--------+--------+-------+
| id|cat1|cat2|cat3|cat4|cat5|cat6|cat7|cat8|cat9|cat10|cat11|cat12|cat13|cat14|cat15|cat16|cat17|cat18|cat19|cat20|cat21|cat22|cat23|cat24|cat25|cat26|cat27|c

In [7]:
train_data.printSchema()

root
 |-- id: integer (nullable = true)
 |-- cat1: string (nullable = true)
 |-- cat2: string (nullable = true)
 |-- cat3: string (nullable = true)
 |-- cat4: string (nullable = true)
 |-- cat5: string (nullable = true)
 |-- cat6: string (nullable = true)
 |-- cat7: string (nullable = true)
 |-- cat8: string (nullable = true)
 |-- cat9: string (nullable = true)
 |-- cat10: string (nullable = true)
 |-- cat11: string (nullable = true)
 |-- cat12: string (nullable = true)
 |-- cat13: string (nullable = true)
 |-- cat14: string (nullable = true)
 |-- cat15: string (nullable = true)
 |-- cat16: string (nullable = true)
 |-- cat17: string (nullable = true)
 |-- cat18: string (nullable = true)
 |-- cat19: string (nullable = true)
 |-- cat20: string (nullable = true)
 |-- cat21: string (nullable = true)
 |-- cat22: string (nullable = true)
 |-- cat23: string (nullable = true)
 |-- cat24: string (nullable = true)
 |-- cat25: string (nullable = true)
 |-- cat26: string (nullable = true)
 |-- ca

In [8]:
train_data.count()

188318

In [9]:
train_data = train_data.withColumnRenamed("loss","label")

In [10]:
[trainingData, validationData] = train_data.randomSplit([0.7, 0.3])

In [11]:
#Prevent loading of data again & again
trainingData.cache()
validationData.cache()

DataFrame[id: int, cat1: string, cat2: string, cat3: string, cat4: string, cat5: string, cat6: string, cat7: string, cat8: string, cat9: string, cat10: string, cat11: string, cat12: string, cat13: string, cat14: string, cat15: string, cat16: string, cat17: string, cat18: string, cat19: string, cat20: string, cat21: string, cat22: string, cat23: string, cat24: string, cat25: string, cat26: string, cat27: string, cat28: string, cat29: string, cat30: string, cat31: string, cat32: string, cat33: string, cat34: string, cat35: string, cat36: string, cat37: string, cat38: string, cat39: string, cat40: string, cat41: string, cat42: string, cat43: string, cat44: string, cat45: string, cat46: string, cat47: string, cat48: string, cat49: string, cat50: string, cat51: string, cat52: string, cat53: string, cat54: string, cat55: string, cat56: string, cat57: string, cat58: string, cat59: string, cat60: string, cat61: string, cat62: string, cat63: string, cat64: string, cat65: string, cat66: string, 

## Build a pipeline
* There are many categorical cols, starts with cat* in this data
* All cont data starts with cont

In [13]:
columns = train_data.columns

In [16]:
#Gets all the categorical cols
cat_cols = list(filter(lambda s: s.startswith("cat"), columns))

In [18]:
cont_cols = list(filter(lambda s: s.startswith("cont"), columns))

In [20]:
#We have to create list of stringindexer, one stringindexer for different col

In [21]:
stringIndexList = []
import pyspark.ml.feature as ft

In [23]:
for col in cat_cols:
    stringIndexList.append(ft.StringIndexer(inputCol=col, outputCol='idx'+col[3:]))

In [28]:
a = stringIndexList[3]

In [29]:
a.getOutputCol()

'idx4'

In [30]:
len(stringIndexList)

116

In [35]:
idx_cat_cols = list(map(lambda s: s.replace('cat','idx'), cat_cols))


In [31]:
from pyspark.ml import Pipeline, PipelineModel

In [36]:
assembler = ft.VectorAssembler(inputCols = idx_cat_cols+cont_cols, outputCol="features")

In [38]:
from pyspark.ml.regression import RandomForestRegressor
estimator = RandomForestRegressor(numTrees=3,maxDepth=5, featuresCol="features", labelCol="label") 

In [41]:
stages = []
stages.extend(stringIndexList)
stages.append(assembler)
stages.append(estimator)

In [42]:
pipeline = Pipeline(stages=stages)

In [ ]:
model = pipeline.fit(trainingData)